## 1. bost-dtn | Throughput Analysis

In [1]:
# import os
# import json
# import numpy as np
# from pathlib import Path

# rootdir = "/Users/eashan22/Desktop/Internship 2021/bbrv2/Brian's Project/bost-dtn"
# fileList = []
# c = 0

# for root, directories, files in os.walk(rootdir):
#     for file in files:
#         if file.endswith("json"):
#             # print(os.path.join(root,file))
#             fileList.append(os.path.join(root,file))
#             c+=1

# print(f"Total files: {c}")

# cubic, bbr2, bbr = [], [], []
# cubic_, bbr2_, bbr_ = [], [], []
# key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

# for i,f in enumerate(fileList):
#     try:
#         path = Path(f)
#         # The MongoDB JSON dump has one object per line, so this works for me.
#         data = [json.loads(line) for line in open(f, 'r')]

#         for i in range(len(data)):
#             if key1 in data[i].keys():
#                 if data[i]['cubic_data_segs']!=0:
#                     cubic.append(data[i]['cubic_data_segs'])
#                     cubic_.append(( path.parent.name, data[i]['cubic_data_segs']) )
#                 elif data[i]['bbr2_data_segs']!=0:
#                     bbr2.append(data[i]['bbr2_data_segs'])
#                     bbr2_.append(( path.parent.name, data[i]['bbr2_data_segs']) )
#                 elif data[i]['bbr_data_segs']!=0:
#                     bbr.append(data[i]['bbr_data_segs'])
#                     bbr_.append(( path.parent.name, data[i]['bbr_data_segs']) )

#     except:
#         print(f"Index: {i}, Parent: {path.parent.name}")

# print(f"cubic:{len(cubic):^5}| bbr2:{len(bbr2):^5}| bbr{len(bbr):^5}")

In [2]:
# cubic_mean  = np.mean(cubic) 
# cubic_mean2 = sum([x[1] for x in cubic_])/len(cubic_)

# bbr2_mean   = np.mean(bbr2) 
# bbr2_mean2  = sum([y[1] for y in bbr2_ ])/len(bbr2_)

# bbr_mean    = np.mean(bbr)
# bbr_mean2   = sum([z[1] for z in bbr_  ])/len(bbr_)

# cubic_var  = np.var(cubic)
# cubic_var2 = sum((x[1]-cubic_mean2)**2 for x in cubic_)/len(cubic_)

# bbr2_var  = np.var(bbr2)
# bbr2_var2  = sum((x[1]-bbr2_mean2)**2 for x in bbr2_)/len(bbr2_)

# bbr_var   = np.var(bbr)
# bbr_var2  = sum((x[1]-bbr_mean2)**2 for x in bbr_)/len(bbr_)

# print("--Mean--")
# print(f"cubic:{cubic_mean:.3f}\nbbr2:{bbr2_mean:.3f}\nbbr:{bbr_mean:.3f}")
# print("\n--Variance--")
# print(f"cubic:{cubic_var:.3f}\nbbr2:{bbr2_var:.3f}\nbbr:{bbr_var:.3f}")

In [3]:
# temp = [x[0] for x in cubic_] 
# cubic_unique = list(set(temp))
# cubic_unique

In [4]:
# bbr_both_p16, both_p16, both_p16_bbr1, cubic_p1 = [], [], [], []

# for c in cubic_:
#     if c[0] == 'pscheduler_bbr_both_p16':
#         bbr_both_p16.append(c[1])

#     elif c[0] == 'pscheduler_both_p16':
#         both_p16.append(c[1])

#     elif c[0] == 'pscheduler_both_p16_bbr1':
#         both_p16_bbr1.append(c[1])

#     elif c[0] == 'pscheduler_cubic_p1':
#         cubic_p1.append(c[1])

# bbr_both_p16_mean, bbr_both_p16_var = np.mean(bbr_both_p16), np.var(bbr_both_p16)
# print(f"{bbr_both_p16_mean:.3f}, {bbr_both_p16_var:.3f}")

# both_p16_mean, both_p16_var = np.mean(both_p16), np.var(both_p16)
# print(f"{both_p16_mean:.3f}, {both_p16_var:.3f}")

# both_p16_bbr1_mean, both_p16_bbr1_var = np.mean(both_p16_bbr1), np.var(both_p16_bbr1)
# print(f"{both_p16_bbr1_mean:.3f}, {both_p16_bbr1_var:.3f}")

# cubic_p1_mean, cubic_p1_var = np.mean(cubic_p1), np.mean(cubic_p1)
# print(f"{cubic_p1_mean:.3f}, {cubic_p1_var:.3f}")

In [5]:
# import os
# import json
# import numpy as np
# from pathlib import Path

# rootdir = "/Users/eashan22/Desktop/Internship 2021/bbrv2/Brian's Project/bost-dtn"
# fileList = []
# c = 0

# for root, directories, files in os.walk(rootdir):
#     for file in files:
#         if file.endswith("json"):
#             fileList.append(os.path.join(root,file))
#             c+=1

# print(f"Total files: {c}")

# cubic, bbr2, bbr = [], [], []
# c_throughput, b2_throughput, b_throughput = [], [], []
# key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

# for i,f in enumerate(fileList):
#     try:
#         path = Path(f)
#         # The MongoDB JSON dump has one object per line, so this works for me.
#         data = [json.loads(line) for line in open(f, 'r')]

#         for i in range(len(data)):
#             if key1 in data[i].keys():
#                 if data[i][key1]!=0:
#                     # cubic.append(data[i]['cubic_data_segs'])
#                     cubic.append(( path.parent.name, data[i][key1]) )
#                     tput_c = (data[i][key1]/(9000*data[i]['p50_rtt']))
#                     c_throughput.append( tput_c )

#                 elif data[i][key2]!=0:
#                     # bbr2.append(data[i][key2])
#                     bbr2.append(( path.parent.name, data[i][key2]) )
#                     tput_b2 = (data[i][key2]/(9000*data[i]['p50_rtt']))
#                     b2_throughput.append( tput_b2 )

#                 elif data[i][key3]!=0:
#                     # bbr.append(data[i][key3])
#                     bbr.append(( path.parent.name, data[i][key3]) )
#                     tput_b = (data[i][key3]/(9000*data[i]['p50_rtt']))
#                     b_throughput.append( tput_b )

#     except:
#         print(f"Index: {i}, Parent: {path.parent.name}")

# print(f"cubic:{len(cubic):^5}| bbr2:{len(bbr2):^5}| bbr{len(bbr):^5}")

In [6]:
# import pandas as pd

# path_cubic, cubic_data_seg = [x[0] for x in cubic], [x[1] for x in cubic]
# path_bbrv2, bbrv2_data_seg  = [x[0] for x in bbr2], [x[1] for x in bbr2]
# path_bbrv, bbrv_data_seg   = [x[0] for x in bbr], [x[1] for x in bbr]

# cubic_df = pd.DataFrame({'cubic':cubic_data_seg, 'path_cubic':path_cubic})
# bbrv2_df = pd.DataFrame({'bbr2':bbrv2_data_seg, 'path_bbrv2':path_bbrv2})
# bbrv_df  = pd.DataFrame({'bbr':bbrv_data_seg, 'path_bbrv':path_bbrv})

In [7]:
# cubic_df.tail(3)

In [8]:
# cubic_df.describe()

In [9]:
# bbrv2_df.describe()

In [10]:
# bbrv_df.describe()

In [11]:
# overall = pd.DataFrame({'mean':[1.327231e+07, 1.066952e+07, 1.147213e+07],
#                         'std' :[2.693767e+07, 7.357977e+06, 7.902758e+06],
#                         'min' :[1.224500e+04, 1.586098e+06, 1.791172e+06],
#                         'max' :[2.627112e+08, 2.660069e+07, 2.746252e+07],
#                         '25%' :[3.411543e+06, 6.556117e+06, 6.523948e+06],
#                         '50%' :[6.660000e+06, 7.914129e+06, 8.107003e+06],
#                         '75%' :[9.332570e+06, 1.385351e+07, 1.489011e+07]
#                         })

# overall

In [12]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,6))

# plt.hist(cubic_data_seg, color='blue')
# plt.hist(bbv2_data_seg, color='red')
# plt.hist(bbrv_data_seg, color='green')

# plt.show()

# Bost-dtn Analysis

In [13]:
import os
import json
import numpy as np
from pathlib import Path

# rootdir = "/Users/eashan22/Desktop/Internship 2021/bbrv2/Brian's Project/bost-dtn"
rootdir = "/home/eadhikarla/Desktop/Brian's Project/bost-dtn"

def traverse(path):
    fileList = []
    c = 0
    for root, directories, files in os.walk(path):
        for file in files:
            if file.endswith("json"):
                fileList.append(os.path.join(root,file))
                c+=1
    print(f"Total files: {c}")
    return fileList

==========================================
## bost-dtn (2021-06-09:14:35)
### 1. bbr2_p1

In [14]:
pscheduler_bbr2_p1  = os.path.join(rootdir, "2021-06-09:14:35/pscheduler_bbr2_p1")
filenames = traverse(pscheduler_bbr2_p1)

tput_bbr2_p1 = []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        # The MongoDB JSON dump has one object per line, so this works for me.
        data = [json.loads(line) for line in open(f, 'r')]
        
        bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            if "interval" in d.keys() and i==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key2 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                bbr2_data_seg = data[j][key2]
                throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)
                tput_bbr2_p1.append( throughput )
        # print(f"Throughput: {(throughput/1e9):.4f} Gbps/sec")
    
    except Exception as e:
        print(e)

print(f"Mean: {np.mean(tput_bbr2_p1)*9.31e-10}  | Variance: {np.var(tput_bbr2_p1)*9.31e-10}")

Total files: 105
float division by zero
Mean: 2.0925770761598767e-07  | Variance: 8.906376684440232e-06


## bost-dtn (2021-06-09:14:35)
### 2. cubic_p1

In [15]:
pscheduler_cubic_p1 = os.path.join(rootdir, "2021-06-09:14:35/pscheduler_cubic_p1")
filenames = traverse(pscheduler_cubic_p1)

tput_p1_cubic = []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        # The MongoDB JSON dump has one object per line, so this works for me.
        data = [json.loads(line) for line in open(f, 'r')]
        
        bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            if "interval" in d.keys() and i==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key1 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                bbr2_data_seg = data[j][key1]
                throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)
                tput_p1_cubic.append( throughput )
        # print(f"Throughput: {(throughput/1e9):.4f} Gbps/sec")
    
    except Exception as e:
        print(e)

print(f"Mean: {np.mean(tput_p1_cubic)*9.31e-10}  | Variance: {np.var(tput_p1_cubic)*9.31e-10}")

Total files: 105
float division by zero
Mean: 2.269248056668279e-07  | Variance: 1.1042407481302774e-05


## bost-dtn (2021-06-09:14:35)
### 3. both_p16  -----> Fig. 4b. in paper

In [16]:
pscheduler_both_p16 = os.path.join(rootdir, "2021-06-09:14:35/pscheduler_both_p16")
filenames = traverse(pscheduler_both_p16)

tput_cubic_p16, tput_bbr2_p16 = [], []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        data = [json.loads(line) for line in open(f, 'r')]
        
        throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        for i,d in zip(range(len(data)),data):
            try:
                if "interval" in d.keys() and i==0:
                    start_time = d['interval']['time'] # Start time of the test
                if "streams" in d.keys():
                    end_time = data[i-1]['interval']['time'] # End time of the test
                    mss = data[i-1]['interval']['mss'] # maximum segment size
                    
                    cubic_data_seg_list, bbr2_data_seg_list = [], []
                    for j in range(len(d['streams'])):
                        if "cubic" in d['streams'][j]['cc']:
                            cubic_data_seg_list.append(d['streams'][j]['data_segs'])
                        elif "bbr2" in d['streams'][j]['cc']:
                            bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                    
                    throughput_cubic = (sum(cubic_data_seg_list)*mss*8)/(end_time-start_time)
                    tput_cubic_p16.append( throughput_cubic )

                    throughput_bbr2 = (sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time)
                    tput_bbr2_p16.append( throughput_bbr2 )

            except Exception as e:
                print(e)

    except Exception as e:
        print(e)

print(f"Cubic Mean: {np.mean(tput_cubic_p16)*9.31e-10:.4f}  | Cubic Variance: {np.var(tput_cubic_p16)*9.31e-10:.4f}")
print(f"Bbrv2 Mean: {np.mean(tput_bbr2_p16)*9.31e-10:.4f}  | Bbrv2 Variance: {np.var(tput_bbr2_p16)*9.31e-10:.4f}")

Total files: 109
Cubic Mean: 4.0103  | Cubic Variance: 3747665639.8610
Bbrv2 Mean: 3.1851  | Bbrv2 Variance: 1570301724.1536


==========================================
## bost-dtn (2021-06-15:02:12)
### 1. bbr2_p1

In [17]:
pscheduler_bbr2_p1  = os.path.join(rootdir, "2021-06-15:02:12/pscheduler_bbr2_p1")
filenames = traverse(pscheduler_bbr2_p1)

tput_bbr2_p1 = []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        data = [json.loads(line) for line in open(f, 'r')]
        
        bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            if "interval" in d.keys() and i==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key2 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                bbr2_data_seg = data[j][key2]
                throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)
                tput_bbr2_p1.append( throughput )
    
    except Exception as e:
        print(e)

print(f"Mean: {np.mean(tput_bbr2_p1)*9.31e-10}  | Variance: {np.var(tput_bbr2_p1)*9.31e-10}")

Total files: 45
float division by zero
Mean: 2.388578878395818e-07  | Variance: 1.0373092530293932e-05


## bost-dtn (2021-06-15:02:12)
### 2. cubic_p1

In [18]:
pscheduler_cubic_p1 = os.path.join(rootdir, "2021-06-15:02:12/pscheduler_cubic_p1")
filenames = traverse(pscheduler_cubic_p1)

tput_p1_cubic = []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        # The MongoDB JSON dump has one object per line, so this works for me.
        data = [json.loads(line) for line in open(f, 'r')]
        
        bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            if "interval" in d.keys() and i==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key1 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                bbr2_data_seg = data[j][key1]
                throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)
                tput_p1_cubic.append( throughput )
        # print(f"Throughput: {(throughput/1e9):.4f} Gbps/sec")
    
    except Exception as e:
        print(e)

print(f"Mean: {np.mean(tput_p1_cubic)*9.31e-10}  | Variance: {np.var(tput_p1_cubic)*9.31e-10}")

Total files: 47
float division by zero
Mean: 2.3386923435703782e-07  | Variance: 1.1819421012832947e-05


## bost-dtn (2021-06-15:02:12)
### 3. both_p16

In [19]:
pscheduler_both_p16 = os.path.join(rootdir, "2021-06-15:02:12/pscheduler_both_p16")
filenames = traverse(pscheduler_both_p16)

tput_cubic_p16, tput_bbr2_p16 = [], []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        data = [json.loads(line) for line in open(f, 'r')]
        
        throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        for i,d in zip(range(len(data)),data):
            try:
                if "interval" in d.keys() and i==0:
                    start_time = d['interval']['time'] # Start time of the test
                if "streams" in d.keys():
                    end_time = data[i-1]['interval']['time'] # End time of the test
                    mss = data[i-1]['interval']['mss'] # maximum segment size
                    
                    cubic_data_seg_list, bbr2_data_seg_list = [], []
                    for j in range(len(d['streams'])):
                        if "cubic" in d['streams'][j]['cc']:
                            cubic_data_seg_list.append(d['streams'][j]['data_segs'])
                        elif "bbr2" in d['streams'][j]['cc']:
                            bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                    
                    throughput_cubic = (sum(cubic_data_seg_list)*mss*8)/(end_time-start_time)
                    tput_cubic_p16.append( throughput_cubic )

                    throughput_bbr2 = (sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time)
                    tput_bbr2_p16.append( throughput_bbr2 )

            except Exception as e:
                print(e)

    except Exception as e:
        print(e)

print(f"Cubic Mean: {np.mean(tput_cubic_p16)*9.31e-10:.4f}  | Cubic Variance: {np.var(tput_cubic_p16)*9.31e-10:.4f}")
print(f"Bbrv2 Mean: {np.mean(tput_bbr2_p16)*9.31e-10:.4f}  | Bbrv2 Variance: {np.var(tput_bbr2_p16)*9.31e-10:.4f}")

Total files: 50
Cubic Mean: 2.1981  | Cubic Variance: 3772052442.1765
Bbrv2 Mean: 5.4003  | Bbrv2 Variance: 3747894822.6419


==========================================
## bost-dtn (2021-06-16:02:12)
### 1. bbr2_p1

In [20]:
pscheduler_bbr2_p1  = os.path.join(rootdir, "2021-06-16:02:12/pscheduler_bbr2_p1")
filenames = traverse(pscheduler_bbr2_p1)

tput_bbr2_p1 = []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        data = [json.loads(line) for line in open(f, 'r')]
        
        bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            if "interval" in d.keys() and i==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key2 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                bbr2_data_seg = data[j][key2]
                throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)
                tput_bbr2_p1.append( throughput )
    
    except Exception as e:
        print(e)

print(f"Mean: {np.mean(tput_bbr2_p1)*9.31e-10}  | Variance: {np.var(tput_bbr2_p1)*9.31e-10}")

Total files: 55
float division by zero
Mean: 2.202604800607542e-07  | Variance: 1.0475994041066544e-05


## bost-dtn (2021-06-16:02:12)
### 2. cubic_p1

In [21]:
pscheduler_cubic_p1 = os.path.join(rootdir, "2021-06-16:02:12/pscheduler_cubic_p1")
filenames = traverse(pscheduler_cubic_p1)

tput_p1_cubic = []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        # The MongoDB JSON dump has one object per line, so this works for me.
        data = [json.loads(line) for line in open(f, 'r')]
        
        bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
        for j,d in enumerate(data):
            if "interval" in d.keys() and i==0:
                start_time = data[j]['interval']['time'] # Start time of the test
            elif key1 in data[j].keys():
                end_time = data[j-1]['interval']['time'] # End time of the test
                mss = data[j-1]['interval']['mss'] # maximum segment size

                bbr2_data_seg = data[j][key1]
                throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)
                tput_p1_cubic.append( throughput )
        # print(f"Throughput: {(throughput/1e9):.4f} Gbps/sec")
    
    except Exception as e:
        print(e)

print(f"Mean: {np.mean(tput_p1_cubic)*9.31e-10}  | Variance: {np.var(tput_p1_cubic)*9.31e-10}")

Total files: 54
float division by zero
Mean: 2.2086850479781964e-07  | Variance: 1.2285207548054892e-05


## bost-dtn (2021-06-16:02:12)
### 3. both_p16

In [22]:
pscheduler_both_p16 = os.path.join(rootdir, "2021-06-16:02:12/pscheduler_both_p16")
filenames = traverse(pscheduler_both_p16)

tput_cubic_p16, tput_bbr2_p16 = [], []
key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

for i,f in enumerate(filenames):
    try:
        path = Path(f)
        data = [json.loads(line) for line in open(f, 'r')]
        
        throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        for i,d in zip(range(len(data)),data):
            try:
                if "interval" in d.keys() and i==0:
                    start_time = d['interval']['time'] # Start time of the test
                if "streams" in d.keys():
                    end_time = data[i-1]['interval']['time'] # End time of the test
                    mss = data[i-1]['interval']['mss'] # maximum segment size
                    
                    cubic_data_seg_list, bbr2_data_seg_list = [], []
                    for j in range(len(d['streams'])):
                        if "cubic" in d['streams'][j]['cc']:
                            cubic_data_seg_list.append(d['streams'][j]['data_segs'])
                        elif "bbr2" in d['streams'][j]['cc']:
                            bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                    
                    throughput_cubic = (sum(cubic_data_seg_list)*mss*8)/(end_time-start_time)
                    tput_cubic_p16.append( throughput_cubic )

                    throughput_bbr2 = (sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time)
                    tput_bbr2_p16.append( throughput_bbr2 )

            except Exception as e:
                print(e)

    except Exception as e:
        print(e)

print(f"Cubic Mean: {np.mean(tput_cubic_p16)*9.31e-10:.4f}  | Cubic Variance: {np.var(tput_cubic_p16)*9.31e-10:.4f}")
print(f"Bbrv2 Mean: {np.mean(tput_bbr2_p16)*9.31e-10:.4f}  | Bbrv2 Variance: {np.var(tput_bbr2_p16)*9.31e-10:.4f}")

Total files: 51
Cubic Mean: 1.6711  | Cubic Variance: 2946501775.2520
Bbrv2 Mean: 5.7825  | Bbrv2 Variance: 3714884469.5304


# Buffer Analysis